In [33]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import g_key

In [34]:
# Import cities file into the cities_pd DataFrame
cities_pd = pd.read_csv("C:/Users/raney/OneDrive/Desktop/Analysis_Projects/World_Weather_Analysis/Resources/cities.csv")
cities_pd.head()


,City,State
0,New York City,New York
1,Los Angeles,California
2,Chicago,Illinois
3,Houston,Texas
4,Philadelphia,Pennsylvania


In [35]:
# Add columns for lat, lng, airport name, airport address, airport rating.
# Hint: Be sure to specify "" as the initial value for each column.
cities_pd["lat"] = ""
cities_pd["lng"] = ""
cities_pd["airport name"] = ""
cities_pd["airport address"] = ""
cities_pd["airport rating"] = ""

In [36]:
# 1. What are the geocoordinates (latitude/longitude) of Seattle, Washington?
def findgeocord (address):
    params = {"address": address, "key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    # Run request
    response = requests.get(base_url, params=params)
    address_geo = response.json()
    lat = address_geo["results"][0]["geometry"]["location"]["lat"]
    lng = address_geo["results"][0]["geometry"]["location"]["lng"]
    #print(f" {address} has latitude as {lat} and longitude as {lng} ")
    coordinates =  str(lat) + "," + str(lng)
    return coordinates

In [37]:
def findnearest(params):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Run request
    searchentities = requests.get(base_url, params=params).json()
    entity_name = searchentities["results"][0]["name"]
    entity_address = searchentities["results"][0]["vicinity"]
    entity_rating = searchentities["results"][0]["rating"]
    print(f" Nearest {search_type} is {entity_name} and address is {entity_address}")
    name_address_rating = f"{entity_name}~{entity_address}~{entity_rating}"
    return name_address_rating
    

In [38]:
# Loop through the cities_pd and get the lat/long for each city
# Hint: `requests.get(target_url).json()`
# Hint: `for index, row in cities_pd.iterrows():`
# Hint: `cities_pd.loc`

search_type = "airport"
keyword =  "international airport"
radius = 50000
    
# use iterrows to iterate through pandas dataframe
for index, row in cities_pd.iterrows():
    city = row["City"]
    state = row["State"]
    address = f"{city},{state}"
    coordinates = findgeocord(address)
    cities_pd.loc[index,"lat"] = coordinates.split(",")[0]
    cities_pd.loc[index,"lng"] = coordinates.split(",")[1]
    params = {
    "radius": radius,
    "types": search_type,
    "keyword": keyword,
    "key": g_key,
    "location": coordinates
    }
    try:
        name_address_rating= findnearest(params)   
        cities_pd.loc[index,"airport name"] = name_address_rating.split("~")[0]
        cities_pd.loc[index,"airport address"] = name_address_rating.split("~")[1]
        cities_pd.loc[index,"airport rating"] = name_address_rating.split("~")[2]
    except (KeyError, IndexError):        
        print(f" Airport missing in {address}")
    

 Nearest airport is John F. Kennedy International Airport and address is Queens
 Nearest airport is Los Angeles International Airport and address is 1 World Way, Los Angeles
 Nearest airport is O'Hare International Airport and address is 10000 W O'Hare Ave, Chicago
 Nearest airport is George Bush Intercontinental Airport and address is 2800 N Terminal Rd, Houston
 Nearest airport is Philadelphia International Airport and address is 8000 Essington Ave, Philadelphia
 Nearest airport is Phoenix Sky Harbor International Airport and address is 3400 E Sky Harbor Blvd, Phoenix
 Nearest airport is San Antonio International Airport and address is 9800 Airport Blvd, San Antonio
 Nearest airport is San Diego International Airport and address is 3225 N Harbor Dr, San Diego
 Nearest airport is Dallas/Fort Worth International Airport and address is 2400 Aviation Dr, DFW Airport


In [39]:
# Use the lat/lng we recovered to identify airports
# Hint: Code is similar to above block (i.e. you will need to use cities_pd.iterrows() again)
# Hint: Use the keyword search for "international+airport"
# Hint: Use a try-except to handle airports that are missing ratings

cities_pd

,City,State,lat,lng,airport name,airport address,airport rating
0,New York City,New York,40.7127753,-74.0059728,John F. Kennedy International Airport,Queens,4
1,Los Angeles,California,34.0522342,-118.2436849,Los Angeles International Airport,"1 World Way, Los Angeles",3.9
2,Chicago,Illinois,41.8781136,-87.6297982,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",4
3,Houston,Texas,29.7604267,-95.3698028,George Bush Intercontinental Airport,"2800 N Terminal Rd, Houston",4.2
4,Philadelphia,Pennsylvania,39.9525839,-75.1652215,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.9
5,Phoenix,Arizona,33.4483771,-112.0740373,Phoenix Sky Harbor International Airport,"3400 E Sky Harbor Blvd, Phoenix",4.3
6,San Antonio,Texas,29.4241219,-98.49362819999999,San Antonio International Airport,"9800 Airport Blvd, San Antonio",4.1
7,San Diego,California,32.715738,-117.1610838,San Diego International Airport,"3225 N Harbor Dr, San Diego",4.3
8,Dallas,Texas,32.7766642,-96.79698789999999,Dallas/Fort Worth International Airport,"2400 Aviation Dr, DFW Airport",4.1


In [41]:
# Save Data to csv
cities_pd.to_csv("C:/Users/raney/OneDrive/Desktop/Analysis_Projects/World_Weather_Analysis/Resources/Airport_Output.csv")

# Visualize to confirm airport data appears